In [1]:
import pandas as pd
import numpy as np
import pandarallel as pdl

In [2]:
# https://pymatgen.org/pymatgen.io.vasp.html

from pymatgen.io.vasp.outputs import Chgcar as chg
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [3]:
from mp_api.client import MPRester
from emmet.core.summary import HasProps
import json
from tqdm import tqdm
import os


my_api_key = "pdDir9sfmf4JPMKDSJzF8W7ZJ7CyqYHE"
# file_path = "../mpids_downloaded.json"
file_path = "../mpids_new.json"

with open(file_path, 'r') as file:
    mpids = json.load(file)

chgcar_path = "../mixed_data"
saved_files = os.listdir(chgcar_path)
saved_files_wo_ext = []
for i in saved_files:
    saved_files_wo_ext.append(i.split(".")[0])

mpids = saved_files_wo_ext

print("Len mpids:",len(mpids))

Len mpids: 16100


In [4]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=32)

INFO: Pandarallel will run on 32 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [5]:
mpids_pd = pd.Series(mpids)

def get_stat(mpid):
    try:
        file_path = "../mixed_data/"+mpid+".chgcar"
        chgcar = chg.from_file(file_path)
        struc = chgcar.structure
        a,b,c = struc.lattice.abc
        grid_x,grid_y,grid_z = chgcar.dim
        n_atoms = struc.num_sites
    except:
        a,b,c = np.nan,np.nan,np.nan
        grid_x,grid_y,grid_z = np.nan,np.nan,np.nan
        n_atoms = np.nan

    return pd.Series([a,b,c,(a+b+c)/3,grid_x,grid_y,grid_z,(grid_x*grid_y*grid_z)**(1/3), n_atoms],
                     index=["a","b","c","avg","grid_x","grid_y","grid_z","grid","n_atoms"]
                     )

In [6]:
mpids_stat = mpids_pd.parallel_apply(get_stat)

In [1]:
mpids_stat.to_csv("mpids_stat.csv")

NameError: name 'mpids_stat' is not defined

In [ ]:
df.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)

In [25]:
chgcar.data["total"].shape

(80, 108, 112)

In [37]:
dir(chgcar.structure.lattice)

['REDIRECT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__get_validators__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__modify_schema__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_calculate_lll',
 '_diags',
 '_generic_json_schema',
 '_inv_matrix',
 '_lll_inverse',
 '_lll_matrix_mappings',
 '_matrix',
 '_pbc',
 '_validate_monty',
 'a',
 'abc',
 'alpha',
 'angles',
 'as_dict',
 'b',
 'beta',
 'c',
 'copy',
 'cubic',
 'd_hkl',
 'dot',
 'find_all_mappings',
 'find_mapping',
 'from_dict',
 'from_parameters',
 'gamma',
 'get_all_distances',
 'get_brillouin_zone',
 'get_cartesian_coords',
 'get_distance_and_image',
 'get_frac_coords_from_lll',
 'get_fractional_coords',
 'get_lll_fr

In [42]:
chgcar.structure.lattice.abc

(5.130117, 6.817619831645058, 7.352613036399236)

In [8]:
for mpid in tqdm(mpids):
    file_path = "../data/"+mpid+".chgcar"
    chgcar = chg.from_file(file_path)
    # lattice_type = SpacegroupAnalyzer(chgcar.structure).get_lattice_type()
    # lattice_types_dict[mpid] = lattice_type

  0%|          | 33/117535 [01:10<69:21:35,  2.13s/it]


KeyboardInterrupt: 